# Example api

In [1]:
import os
os.chdir('../AdeleCV')

In [2]:
os.getcwd()

'D:\\ds\\AdeleCV\\AdeleCV'

In [3]:
from api.logs import enable_logs
from api.data.segmentations import SegmentationDataset
from logging import StreamHandler

In [4]:
enable_logs(StreamHandler())

In [ ]:
from api.config import Settings
from pathlib import Path

In [ ]:
# you can change the path to save logs and weights
# Settings.update_tmp_path(Path('your_path'))

In [5]:
from api.data.segmentations.types import ImageMask

In [6]:
segm_dataset = SegmentationDataset(
    r'F:\dataset\ph2',
    ImageMask, # аннотация для наследника
    (256, 256),
    (0.7, 0.2, 0.1),
    16,
)

 100% |█████████████████| 200/200 [1.3s elapsed, 0s remaining, 154.4 samples/s]         


INFO - Split dataset train size: 140, valid size: 40, test size: 20
INFO - Creating a dataset
DEBUG - Dataset created with params, dataset dir: F:\dataset\ph2, classes: {0: 'background', 1: 'target'}, batch size: 16


In [7]:
from api.models.segmentations import  get_encoders, get_pretrained_weights, get_models, get_torch_optimizers, get_losses, get_optimize_scores
from api.optimize.segmentations import get_hp_optimizers
# создать отдельный модуль для получения возможных параметров

In [8]:
print('models: ', get_models())
print('encoders: ', get_encoders())
print('pretrained_weights', get_pretrained_weights())
print('optimizers', get_torch_optimizers())
print('losses', get_losses())
print('hp optimizers', get_hp_optimizers())
print('optimize scores', get_optimize_scores())

models:  ['Unet', 'UnetPlusPlus', 'MAnet', 'Linknet', 'FPN', 'PSPNet', 'DeepLabV3', 'DeepLabV3Plus', 'PAN']
encoders:  ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x4d', 'resnext101_32x8d', 'resnext101_32x16d', 'resnext101_32x32d', 'resnext101_32x48d', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn107', 'dpn131', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'inceptionresnetv2', 'inceptionv4', 'efficientnet-b0', 'efficientnet-b1', 'efficientnet-b2', 'efficientnet-b3', 'efficientnet-b4', 'efficientnet-b5', 'efficientnet-b6', 'efficientnet-b7', 'mobilenet_v2', 'xception', 'timm-efficientnet-b0', 'timm-efficientnet-b1', 'timm-efficientnet-b2', 'timm-efficientnet-b3', 'timm-efficientnet-b4', 'timm-efficientnet-b5', 'timm-efficientnet-b6', 'timm-e

In [9]:
from api.optimize.segmentations import HyperParamsSegmentation
from api.optimize.segmentations import HPOptimizer

In [20]:
params = HyperParamsSegmentation(
    strategy='TPESampler',
    architectures=['UnetPlusPlus'],
    encoders=['mobilenet_v2'],
    pretrained_weights=['imagenet'],
    loss_fns=['JaccardLoss'],
    optimizers=['AdamW', 'RMSprop'],
    epoch_range=(1, 1),
    lr_range=(0.001, 0.003),
    optimize_score='loss',
)

In [21]:
hp_optimizer = HPOptimizer(
    hyper_params=params,
    num_trials=1,
    device='GPU',
    dataset=segm_dataset
)

DEBUG - Create hp optimizer with params: strategy: TPESampler num_trials: 1 num_classes: 2 device: cuda hps: {'strategy': 'TPESampler', 'architectures': ['UnetPlusPlus'], 'encoders': ['mobilenet_v2'], 'pretrained_weights': ['imagenet'], 'loss_fns': ['JaccardLoss'], 'optimizers': ['AdamW', 'RMSprop'], 'epoch_range': (1, 1), 'lr_range': (0.001, 0.003), 'optimize_score': 'loss'}


In [12]:
from api.config import Settings

In [13]:
tensorboard_logs_path = Settings.TENSORBOARD_LOGS_PATH.as_posix()

In [14]:
tensorboard_logs_path

'tmp/e8efb598d989494684c6f19959234443/tensorboard'

In [15]:
%reload_ext tensorboard

In [29]:
tensorboard_logs_path

'tmp/e8efb598d989494684c6f19959234443/tensorboard'

In [36]:
!tensorboard --logdir $tensorboard_logs_path

^C


In [22]:
hp_optimizer.optimize()

INFO - Train models started
[I 2023-02-23 01:09:25,742] A new study created in memory with name: no-name-33f8c4b5-992d-4ebc-bb6b-ba6c396f6210
DEBUG - Create tensorboard logger, path: tmp/e8efb598d989494684c6f19959234443/tensorboard/cd323ae
DEBUG - Dataset updated
INFO - Model cd323ae_UnetPlusPlus_mobilenet_v2_imagenet_RMSprop_JaccardLoss_lr=0,002801821172700003 trained with test loss 0.8617084324359894
DEBUG - Add predictions for model: cd323ae_UnetPlusPlus_mobilenet_v2_imagenet_RMSprop_JaccardLoss_lr=0,002801821172700003


 200 [9.6s elapsed, 23.0 samples/s]   


DEBUG - Save weights model: cd323ae_UnetPlusPlus_mobilenet_v2_imagenet_RMSprop_JaccardLoss_lr=0,002801821172700003, path: tmp/e8efb598d989494684c6f19959234443/weights/cd323ae.pt
[I 2023-02-23 01:09:46,868] Trial 0 finished with value: 0.788293977578481 and parameters: {'optimizer': 'RMSprop', 'architecture': 'UnetPlusPlus', 'encoders': 'mobilenet_v2', 'pretrained_weight': 'imagenet', 'lr': 0.002801821172700003, 'loss': 'JaccardLoss', 'num_epoch': 1}. Best is trial 0 with value: 0.788293977578481.
INFO - Study statistics:
INFO - Number of finished trials: 1
INFO - Number of pruned trials: 0
INFO - Number of complete trials: 1
INFO - Best trial:
INFO - Value: 0.788293977578481
INFO - Params: 
INFO - optimizer: RMSprop
INFO - architecture: UnetPlusPlus
INFO - encoders: mobilenet_v2
INFO - pretrained_weight: imagenet
INFO - lr: 0.002801821172700003
INFO - loss: JaccardLoss
INFO - num_epoch: 1
INFO - Train models is over


In [37]:
hp_optimizer.stats_models

,_id,name,architecture,encoder,pretrained_weight,lr,optimizer,loss_fn,num_epoch,loss,fbeta_score,f1_score,iou_score,accuracy,positive_predictive_value,sensitivity
0,cd323ae,cd323ae_UnetPlusPlus_mobilenet_v2_imagenet_RMS...,UnetPlusPlus,mobilenet_v2,imagenet,0.002802,RMSprop,JaccardLoss,1,0.862,0.185,0.185,0.121,0.241,0.62,0.5


In [42]:
from api.modification_models import ExportWeights

In [ ]:
# you can save the selected weights in a zip file (feature for ui)
# ExportWeights().create_zip()

In [44]:
import fiftyone as fo

In [ ]:
fo.launch_app(segm_dataset.fo_dataset)